In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pls
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
df = pd.read_excel("TemizlenmisTerminalDatasi/foms17_1yeardata.xlsx")
df['sonuc'] = 100
#np.random.randint(99, 102, size=len(df))

df.head()


,ID,KULLANICI,EKİP,ÖLÇÜM NEDENİ,İSTASYON,YAKIT TÜRÜ,BATTERY,TARİH,SAAT,CH1,...,CH4,CT,ST,ÖLÇÜM,ALAN,PLAKA,FATURA,KOL,NOT,sonuc
0,143-158-132-GL-1-17_1417,ayla.unal@opet.com.tr,TDE Kocaeli,Terminal Ölçümü / Müşteri Şikayeti / Müşteri N...,METEOR PETROL ÜRÜNLERİ SANAYİVE TİCARET ANONİM...,Diesel UlltraForce,8.46,2022-06-14,14:14:00,25956,...,7973,25.5,29.6,-9.5,NaN,34 aoa 875,NaN,NaN,NaN,100
1,143-158-132-GL-1-17_2445,ayla.unal@opet.com.tr,TDE Kocaeli,Terminal Ölçümü / Tanker Kontrolü,AKPA DAYANIKLI TÜKETİM LPG VEAKRY.ÜRN. PAZ. A.Ş.,Diesel EcoForce,8.41,2022-10-17,11:05:00,26486,...,9450,21.3,25.8,-12.3,AKIN göl,33dce52,10/13/2022,203.0,NaN,100
2,143-158-132-GL-1-17_2344,ayla.unal@opet.com.tr,TDE Kocaeli,Terminal Ölçümü / Tanker Kontrolü,ATLI OTOMOTİV NAKLİYAT SANAYİTİCARET LİMİTED Ş...,Diesel UltraForce,8.43,2022-10-05,13:24:00,29861,...,9059,22.6,26.7,-8.7,M.BALBAL,34 ETE 244,10/03/2022,803.0,NaN,100
3,143-158-132-GL-1-17_2401,ayla.unal@opet.com.tr,TDE Kocaeli,Terminal Ölçümü / Tanker Kontrolü,MOBİPA MOBİLYA TEKS. İNŞ.NAK.PTR.ÜRN.SÜPER MAR...,BENZİN K95 Etanol,8.51,2022-10-11,14:29:00,37175,...,10282,23.0,27.4,-6.0,AKIN GÖL,16PE566,10/08/2022,203.0,NaN,100
4,143-158-132-GL-1-17_2400,ayla.unal@opet.com.tr,TDE Kocaeli,Terminal Ölçümü / Tanker Kontrolü,MOBİPA MOBİLYA TEKS. İNŞ.NAK.PTR.ÜRN.SÜPER MAR...,BENZİN K95 Etanol,8.51,2022-10-11,14:27:00,37574,...,10119,22.9,27.1,-6.0,AKIN GÖL,16 PE 566,10/08/2022,203.0,NaN,100


In [9]:
# 1. Veri Setini Yükleme ve Hazırlama
X = df[['CH1', 'CH2', 'CH3', 'CH4', 'CT', 'ST']].values
y = df['sonuc'].values

# Verileri tensörlere dönüştürme
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# 2. Yapay Sinir Ağı Modelini Oluşturma
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(6, 32)  # Giriş katmanı 6 bağımsız değişken, gizli katman 32 düğüm
        self.fc2 = nn.Linear(32, 1)  # Gizli katman 32 düğüm, çıkış katmanı 1 bağımlı değişken (tahmin)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 3. Kayıp Fonksiyonu ve Optimizasyon Fonksiyonunu Seçin
model = NeuralNetwork()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Modeli Eğitme
num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        


/Users/fatihozkan/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2007])) that is different to the input size (torch.Size([2007, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 1684123.5000
Epoch [200/1000], Loss: 903671.0625
Epoch [300/1000], Loss: 598259.7500
Epoch [400/1000], Loss: 486077.1250
Epoch [500/1000], Loss: 425473.5312
Epoch [600/1000], Loss: 375643.7500
Epoch [700/1000], Loss: 331326.5938
Epoch [800/1000], Loss: 291950.3438
Epoch [900/1000], Loss: 256481.2969
Epoch [1000/1000], Loss: 224010.7500


In [11]:
with torch.no_grad():
    test_data = pd.read_excel('TemizlenmisTerminalDatasi/foms17_motorin_300ppb.xlsx')  # Test verilerini yükleyin
    test_data["sonuc"] = 100
    X_test = test_data[['CH1', 'CH2', 'CH3', 'CH4', 'CT', 'ST']].values
    y_test = test_data['sonuc'].values
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    model.eval()  # Modeli değerlendirme moduna geçirin

    test_predictions = model(X_test)
    test_loss = criterion(test_predictions, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')


Test Loss: 380694.5312


/Users/fatihozkan/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([410])) that is different to the input size (torch.Size([410, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [12]:
# 1. Veri Setini Yükleme ve Hazırlama
X = df[['CH1', 'CH2', 'CH3', 'CH4', 'CT', 'ST']].values
y = df['sonuc'].values
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# 1. Veri Setini ve Yapay Sinir Ağı Modelini Oluşturma
# Örnek veri seti yerine, gerçek veri setinizi yükleyin ve hazırlayın.
# Eğitim ve test verileri olarak ayırın.

# Yapay Sinir Ağı Modelini Tanımlama
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(6, 32)  # Giriş katmanı 6 bağımsız değişken, gizli katman 32 düğüm
        self.fc2 = nn.Linear(32, 1)  # Gizli katman 32 düğüm, çıkış katmanı 1 bağımlı değişken (tahmin)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = NeuralNetwork()

# 4. Modeli Eğitme
num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# 3. Tahminleme İşlemi
def yapay_sinir_agi_tahminle():
    # Kullanıcıdan 6 adet değişkenin değerini alın
    degisken1 = float(input("Degisken1 değerini girin: "))
    degisken2 = float(input("Degisken2 değerini girin: "))
    degisken3 = float(input("Degisken3 değerini girin: "))
    degisken4 = float(input("Degisken4 değerini girin: "))
    degisken5 = float(input("Degisken5 değerini girin: "))
    degisken6 = float(input("Degisken6 değerini girin: "))

    # Giriş değerlerini tensörlere dönüştürme ve model üzerinden geçirme
    with torch.no_grad():
        input_data = torch.tensor([degisken1, degisken2, degisken3, degisken4, degisken5, degisken6], dtype=torch.float32)
        output = model(input_data)

    # Tahmin sonucunu ekrana yazdırma
    print(f"Tahmin sonucu: {output.item()}")

# Kullanıcıdan tahmin yapılmasını isteyin
while True:
    yapay_sinir_agi_tahminle()
    devam_et = input("Yeni bir tahmin yapmak istiyor musunuz? (E/H): ")
    if devam_et.upper() != "E":
        break

/Users/fatihozkan/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2007])) that is different to the input size (torch.Size([2007, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [100/1000], Loss: 3356939520.0000
Epoch [200/1000], Loss: 3356939520.0000
Epoch [300/1000], Loss: 3356939520.0000
Epoch [400/1000], Loss: 3356939520.0000
Epoch [500/1000], Loss: 3356939520.0000
Epoch [600/1000], Loss: 3356939520.0000
Epoch [700/1000], Loss: 3356939520.0000
Epoch [800/1000], Loss: 3356939520.0000
Epoch [900/1000], Loss: 3356939520.0000
Epoch [1000/1000], Loss: 3356939520.0000
Degisken1 değerini girin: 873276
Degisken2 değerini girin: 296870
Degisken3 değerini girin: 549055
Degisken4 değerini girin: 506469
Degisken5 değerini girin: 22.5
Degisken6 değerini girin: 25.9
Tahmin sonucu: -108295.8828125
Yeni bir tahmin yapmak istiyor musunuz? (E/H): h
